<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Normalizando-o-dado" data-toc-modified-id="Normalizando-o-dado-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Normalizando o dado</a></span></li><li><span><a href="#Separando-treino-e-teste" data-toc-modified-id="Separando-treino-e-teste-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Separando treino e teste</a></span></li><li><span><a href="#Rodando-regressão-logistica" data-toc-modified-id="Rodando-regressão-logistica-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Rodando regressão logistica</a></span></li><li><span><a href="#Rodando-logit-com-CV" data-toc-modified-id="Rodando-logit-com-CV-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Rodando logit com CV</a></span></li></ul></div>

In [1]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH, MODEL_PATH

import os
from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

colorscale = ['#025951', '#8BD9CA', '#BF7F30', '#F2C124', '#8C470B', '#DFC27D']

In [2]:
df = pd.read_csv(TREAT_PATH / 'analysis' / 'df_alunos.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182838 entries, 0 to 182837
Data columns (total 19 columns):
ID                            182838 non-null int64
CO_ENTIDADE                   182838 non-null int64
TP_SEXO                       182838 non-null int64
NU_IDADE                      182838 non-null int64
IN_DISTORCAO                  182838 non-null int64
IN_TRANSPORTE_PUBLICO         182838 non-null int64
IN_LOCAL_ESCOLA               182838 non-null int64
N_TURMA                       182838 non-null int64
IN_EVASAO                     182838 non-null int64
NU_ANO_CENSO                  182838 non-null int64
CO_MUNICIPIO                  182838 non-null int64
IN_LABORATORIO_INFORMATICA    182838 non-null float64
IN_LABORATORIO_CIENCIAS       182838 non-null float64
IN_QUADRA_ESPORTES            182838 non-null float64
IN_BIBLIOTECA                 182838 non-null float64
IN_BANHEIRO_FORA_PREDIO       182838 non-null float64
IN_AREA_VERDE                 182838 non-null f

In [3]:
df = df.drop('NU_ANO_CENSO', axis=1)
df.head()

,ID,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,IN_EVASAO,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
0,112132289130,33009627,2,17,0,0,2,0,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
1,113220290328,33009627,2,21,1,0,2,0,1,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
2,112752673352,33009627,2,16,0,0,2,0,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
3,117459545545,33009627,2,17,0,0,2,0,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
4,113868062938,33009627,2,18,1,0,2,0,1,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0


In [4]:
df['IN_EVASAO'].value_counts(normalize=True)

0    0.657172
1    0.342828
Name: IN_EVASAO, dtype: float64

In [5]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149234 entries, 0 to 182836
Data columns (total 18 columns):
ID                            149234 non-null int64
CO_ENTIDADE                   149234 non-null int64
TP_SEXO                       149234 non-null int64
NU_IDADE                      149234 non-null int64
IN_DISTORCAO                  149234 non-null int64
IN_TRANSPORTE_PUBLICO         149234 non-null int64
IN_LOCAL_ESCOLA               149234 non-null int64
N_TURMA                       149234 non-null int64
IN_EVASAO                     149234 non-null int64
CO_MUNICIPIO                  149234 non-null int64
IN_LABORATORIO_INFORMATICA    149234 non-null float64
IN_LABORATORIO_CIENCIAS       149234 non-null float64
IN_QUADRA_ESPORTES            149234 non-null float64
IN_BIBLIOTECA                 149234 non-null float64
IN_BANHEIRO_FORA_PREDIO       149234 non-null float64
IN_AREA_VERDE                 149234 non-null float64
IN_N_COMP_15                  149234 non-null

In [6]:
df['IN_EVASAO'].value_counts(normalize=True)

0    0.676689
1    0.323311
Name: IN_EVASAO, dtype: float64

In [7]:
# Unicidade do ID
df['ID'].value_counts()[:5]

111677686633    1
116834315887    1
112387438468    1
144787363408    1
144782776569    1
Name: ID, dtype: int64

In [8]:
df = df.set_index('ID', drop=True)

In [9]:
X = df.columns
X = X.drop('IN_EVASAO')
df_x = df[X]

df_x.head()

,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
ID,,,,,,,,,,,,,,,,
112132289130,33009627,2,17,0,0,2,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
113220290328,33009627,2,21,1,0,2,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
112752673352,33009627,2,16,0,0,2,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
117459545545,33009627,2,17,0,0,2,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
113868062938,33009627,2,18,1,0,2,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0


In [10]:
y = df['IN_EVASAO']
y.head()

ID
112132289130    0
113220290328    1
112752673352    0
117459545545    0
113868062938    1
Name: IN_EVASAO, dtype: int64

### Normalizando o dado

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df_x)
X_norm = pd.DataFrame(scaler.transform(df_x),columns = df_x.columns)

In [12]:
X_norm.index = df_x.index

In [13]:
X_norm.head()

,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
ID,,,,,,,,,,,,,,,,
112132289130,-1.088033,1.022623,0.180708,-0.665617,-1.500712,1.397555,-1.182787,-1.56678,0.414526,0.517692,0.255455,0.25596,0.509513,-2.326973,1.439747,-0.33438
113220290328,-1.088033,1.022623,2.405032,1.502366,-1.500712,1.397555,-1.182787,-1.56678,0.414526,0.517692,0.255455,0.25596,0.509513,-2.326973,1.439747,-0.33438
112752673352,-1.088033,1.022623,-0.375374,-0.665617,-1.500712,1.397555,-1.182787,-1.56678,0.414526,0.517692,0.255455,0.25596,0.509513,-2.326973,1.439747,-0.33438
117459545545,-1.088033,1.022623,0.180708,-0.665617,-1.500712,1.397555,-1.182787,-1.56678,0.414526,0.517692,0.255455,0.25596,0.509513,-2.326973,1.439747,-0.33438
113868062938,-1.088033,1.022623,0.736789,1.502366,-1.500712,1.397555,-1.182787,-1.56678,0.414526,0.517692,0.255455,0.25596,0.509513,-2.326973,1.439747,-0.33438


In [14]:
X_norm.describe()

,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
count,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05,1.492340e+05
mean,-5.603819e-15,-7.475187e-18,3.712835e-16,-1.238881e-16,-3.618562e-17,-1.668348e-16,4.685086e-17,1.461665e-13,-8.017972e-17,-3.809013e-17,-1.447425e-17,-8.384589e-17,-3.532859e-17,9.179720e-17,1.252213e-16,5.467838e-16
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-1.258762e+00,-9.778771e-01,-2.599699e+00,-6.656167e-01,-1.500712e+00,-1.312952e+00,-1.182787e+00,-2.138671e+00,-2.412391e+00,-1.931652e+00,-3.914588e+00,-3.906855e+00,-1.962660e+00,-2.326973e+00,-6.945665e-01,-5.813171e+00
25%,-4.852776e-01,-9.778771e-01,-3.753736e-01,-6.656167e-01,-1.500712e+00,-1.312952e+00,-1.182787e+00,-8.143248e-01,4.145265e-01,5.176916e-01,2.554547e-01,2.559604e-01,5.095126e-01,4.297428e-01,-6.945665e-01,-3.343798e-01
50%,-2.032829e-01,-9.778771e-01,-3.753736e-01,-6.656167e-01,6.663505e-01,4.230116e-02,2.609915e-01,4.055836e-01,4.145265e-01,5.176916e-01,2.554547e-01,2.559604e-01,5.095126e-01,4.297428e-01,-6.945665e-01,-3.343798e-01
75%,3.847840e-01,1.022623e+00,1.807076e-01,1.502366e+00,6.663505e-01,1.397555e+00,2.609915e-01,6.654197e-01,4.145265e-01,5.176916e-01,2.554547e-01,2.559604e-01,5.095126e-01,4.297428e-01,1.439747e+00,-3.343798e-01
max,8.667493e+00,1.022623e+00,3.354558e+01,1.502366e+00,6.663505e-01,1.397555e+00,1.704770e+00,1.765162e+00,4.145265e-01,5.176916e-01,2.554547e-01,2.559604e-01,5.095126e-01,4.297428e-01,1.439747e+00,7.883807e+00


In [15]:
# clf = LogisticRegression(random_state = 0)
# clf.fit(X_scaled, y)

# feature_importance = abs(clf.coef_[0])
# feature_importance = 100.0 * (feature_importance / feature_importance.max())
# sorted_idx = np.argsort(feature_importance)
# pos = np.arange(sorted_idx.shape[0]) + .5

# featfig = plt.figure()
# featax = featfig.add_subplot(1, 1, 1)
# featax.barh(pos, feature_importance[sorted_idx], align='center')
# featax.set_yticks(pos)
# featax.set_yticklabels(np.array(X.columns)[sorted_idx], fontsize=8)
# featax.set_xlabel('Relative Feature Importance')

### Separando treino e teste

In [16]:
# from sklearn.model_selection import train_test_split 
# X_train, X_test, y_train, y_test = train_test_split(X_norm, df_y, test_size=0.3, random_state=0)

X_train = pd.read_csv(TREAT_PATH / 'modelo' / 'df_treino.csv', index_col='ID')
X_test = pd.read_csv(TREAT_PATH / 'modelo' / 'df_teste.csv', index_col='ID')

In [17]:
X_train.head()

,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL,IN_EVASAO
ID,,,,,,,,,,,,,,,,,
116560246166,-0.418362,1.022623,-0.931455,-0.665617,-1.500712,0.042301,0.260992,-1.914696,0.414526,0.517692,0.255455,0.25596,0.509513,0.429743,-0.694566,-0.33438,0
117459039844,0.969508,1.022623,-0.375374,-0.665617,0.666351,-1.312952,0.260992,0.665420,0.414526,0.517692,0.255455,0.25596,0.509513,0.429743,1.439747,-0.33438,0
114290492435,-0.773354,1.022623,-0.375374,-0.665617,0.666351,1.397555,0.260992,-1.758669,0.414526,0.517692,0.255455,0.25596,0.509513,-2.326973,-0.694566,-0.33438,0
112968002103,0.636104,-0.977877,-0.375374,-0.665617,0.666351,-1.312952,0.260992,0.665420,0.414526,0.517692,0.255455,0.25596,0.509513,0.429743,-0.694566,-0.33438,0
110726113092,1.353912,-0.977877,-0.931455,-0.665617,0.666351,-1.312952,1.704770,0.665420,0.414526,0.517692,0.255455,0.25596,0.509513,0.429743,-0.694566,-0.33438,1


In [18]:
y_train = X_train['IN_EVASAO']
y_train.head()

ID
116560246166    0
117459039844    0
114290492435    0
112968002103    0
110726113092    1
Name: IN_EVASAO, dtype: int64

In [19]:
y_train.value_counts(normalize=True)

0    0.676507
1    0.323493
Name: IN_EVASAO, dtype: float64

In [20]:
y_test = X_test['IN_EVASAO']
y_test.head()

ID
112810788919    0
112378190408    0
117460839999    0
111155952293    0
119561769505    0
Name: IN_EVASAO, dtype: int64

In [21]:
y_test.value_counts(normalize=True)

0    0.677112
1    0.322888
Name: IN_EVASAO, dtype: float64

In [22]:
# # Salvando treino e teste
# X_train['IN_EVASAO'] = y_train
# X_train.to_csv(TREAT_PATH / 'modelo' / 'df_treino.csv')

# X_test['IN_EVASAO'] = y_test
# X_test.to_csv(TREAT_PATH / 'modelo' / 'df_teste.csv')

In [23]:
X_train = X_train.drop('IN_EVASAO', axis=1)
X_test = X_test.drop('IN_EVASAO', axis=1)

### Rodando regressão logistica

In [24]:
# from sklearn.linear_model import LogisticRegression

# logit = LogisticRegression(random_state=0)
# logit.fit(X_train, y_train)

# # Salvando modelo
# filename = 'logit_reg_model.sav'

# import pickle
# pickle.dump(logit, open(filename, 'wb'))
import pickle

filename = 'logit_reg_model.sav'
logit = pickle.load(open(MODEL_PATH / filename,"rb"))

In [25]:
result = logit.score(X_test, y_test)
result

0.7253132608161533

In [26]:
y_pred = logit.predict(X_test)

In [27]:
df_result = pd.DataFrame({'Real': y_test.values.flatten(), 
                          'Predito': y_pred.flatten()})
df_result.head()

,Real,Predito
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [28]:
df_result['Predito'].value_counts()

0    33555
1    11216
Name: Predito, dtype: int64

In [29]:
df_result['Real'].value_counts()

0    30315
1    14456
Name: Real, dtype: int64

In [30]:
print(logit.coef_)

[[ 0.02001104 -0.04057315  0.27586702  0.56425535 -0.156661   -0.07957841
   0.05447396  0.04923563 -0.00255688  0.03257548 -0.01379747  0.01995079
   0.02872926 -0.01591184 -0.01266078 -0.08029667]]


* Normalizando a matriz de confusão

In [31]:
from sklearn.metrics import confusion_matrix

cfm = confusion_matrix(y_test, y_pred)
cfm

array([[25786,  4529],
       [ 7769,  6687]])

In [32]:
cfm.sum(axis=0) # Predito

array([33555, 11216])

In [33]:
cfm.sum(axis=1) # Real

array([30315, 14456])

In [34]:
cfm = cfm.astype(float) / cfm.sum(axis=1)[:, np.newaxis]
cfm

array([[0.85060201, 0.14939799],
       [0.53742391, 0.46257609]])

In [35]:
df_cfm = pd.DataFrame(cfm)
# df_cfm = df_cfm.sort_index(ascending=False)
df_cfm

,0,1
0,0.850602,0.149398
1,0.537424,0.462576


In [36]:
from sklearn.metrics import confusion_matrix

def plot_confusion(test, pred, labels, title):

    cfm = confusion_matrix(test, pred)
    cfm = cfm.astype(float) / cfm.sum(axis=1)[:, np.newaxis]
    
    cfm = pd.DataFrame(cfm).transpose()
    
    x_labels = {'ticktext': list(labels.values()), 'tickvals': list(labels.keys())}
    y_labels = {'autorange': 'reversed', 'ticktext': list(labels.values()), 'tickvals': list(labels.keys())}

    cfm.iplot(kind='heatmap', colorscale='Blues', filename='cufflinks/simple-heatmap', 
             layout={'xaxis': x_labels, 'yaxis': y_labels, 'title': title})

In [37]:
evasao_label = {0: 'Não evadiu', 1: 'Evadiu'}
title = 'Regressão Logística: Real x Predito'

plot_confusion(y_test, y_pred, evasao_label, title)

In [38]:
from sklearn.metrics import recall_score, precision_score

recall_score(y_test, y_pred)

0.46257609297177643

In [39]:
precision_score(y_test, y_pred)

0.5962018544935807

In [40]:
y_score = logit.decision_function(X_test)
y_score

array([-1.74626291, -1.46788518, -1.0162173 , ..., -1.41906155,
       -1.21372703, -0.84383235])

In [41]:
from sklearn.metrics import roc_curve, auc

def plot_roc(y_test, y_score, title):
    
    # Compute ROC curve and ROC area for each class
    fpr, tpr, _ = roc_curve(y_test, y_score, pos_label=1)
    roc_auc = auc(fpr, tpr)

    ref = np.linspace(0,1)

    data = [go.Scatter(x=ref, y=ref, mode='markers', showlegend=False), 
              go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC curve (area = %0.2f)' % roc_auc)]

    layout = dict(xaxis=dict(title='False Positive Rate'), yaxis=dict(title='True Positive Rate'), 
                  title=title)

    fig = go.Figure(data, layout)
    cf.iplot(fig)

In [42]:
plot_roc(y_test, y_score, title='ROC - Logit')

### Rodando logit com CV

In [43]:
# from sklearn.linear_model import LogisticRegressionCV

# logit_cv = LogisticRegressionCV(cv=5, random_state=0)
# logit_cv.fit(X_train, y_train)

# # Salvando modelo
# import pickle
# filename = 'logit_reg_cv_model.sav'

# pickle.dump(logit_cv, open(filename, 'wb'))

import pickle
filename = 'logit_reg_cv_model.sav'
logit_cv = pickle.load(open(MODEL_PATH / filename,"rb"))

In [44]:
result_cv = logit_cv.score(X_test, y_test)
result_cv

0.7257823144446182

In [45]:
y_pred_cv = logit_cv.predict(X_test)

df_result_cv = pd.DataFrame({'Actual': y_test.values.flatten(), 'Predicted': y_pred_cv.flatten()})
df_result_cv.head()

,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [46]:
df_result_cv['Predicted'].value_counts()

0    33948
1    10823
Name: Predicted, dtype: int64

In [47]:
df_result_cv['Actual'].value_counts()

0    30315
1    14456
Name: Actual, dtype: int64

In [48]:
cfm = confusion_matrix(y_test, y_pred_cv)
cfm = cfm.astype(float) / cfm.sum(axis=1)[:, np.newaxis]

In [49]:
cfm

array([[0.85743032, 0.14256968],
       [0.55029054, 0.44970946]])

In [50]:
evasao_label = {0: 'Não evadiu', 1: 'Evadiu'}
title = 'Regressão Logística CV: Real x Predito'

plot_confusion(y_test, y_pred_cv, evasao_label, title)

In [51]:
recall_score(y_test, y_pred_cv)

0.44970946319867183

In [52]:
precision_score(y_test, y_pred_cv)

0.6006652499307031

In [53]:
y_score_cv = logit_cv.decision_function(X_test)
plot_roc(y_test, y_score_cv, 'ROC - Logit CV (k=5)')

In [54]:
# data_final_vars=data_final.columns.values.tolist()
# y=['y']
# X=[i for i in data_final_vars if i not in y]from sklearn.feature_selection import RFE
# from sklearn.linear_model import LogisticRegressionlogreg = LogisticRegression()rfe = RFE(logreg, 20)
# rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
# print(rfe.support_)
# print(rfe.ranking_)